# Predictions with Fully Convolutional Networks
Instead of using purely traditional computer vision methods, I use Deep Learning as it has shown to match or even exceed previous state-of-the-art methods. 

![unet](resources/unet.png)

In [ ]:
from models import *

import numpy as np
from scipy.misc import imresize
import pickle
import imageio

# Dependencies for Model

import matplotlib.pyplot as plt

from skimage.measure import label

from keras.preprocessing import image
from tensorflow import set_random_seed
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max


In [ ]:
sess = tf.Session()
K.set_session(sess)

set_random_seed(0)

In [ ]:
DATA_SPLIT = 650
IMAGE_MEAN = 0.17166166804067506

In [ ]:
images = pickle.load(open('../processed_data/gray_train_X.p', 'rb'))
labels = pickle.load(open('../processed_data/train_Y_baseline.p', 'rb'))

In [ ]:
# Weighted labels
labels = pickle.load(open('../processed_data/weighted_train_masks.p', 'rb'))
weights = pickle.load(open('../processed_data/weighted_train_weights.p', 'rb'))
weights = np.expand_dims(weights, -1)

In [ ]:
print(images.shape)

## Cross-Validation Split
Since the amount of training data is limited ( < 1000), I only used a training and test set instead of also including a validation set. While this does sacrifice the ability to create a more robust model, the greater number of samples will reduce the overall variance of the test set.

In [ ]:
labels = np.expand_dims(labels, axis=-1)

In [ ]:
train_X = images[:DATA_SPLIT,:,:,:] - IMAGE_MEAN
train_Y = labels[:DATA_SPLIT,:,:,:] #Reduced range to [0,1]

valid_X = images[DATA_SPLIT:,:,:,:] - IMAGE_MEAN
valid_Y = labels[DATA_SPLIT:,:,:,:] #Reduced range to [0,1]

train_Y_w = labels[:DATA_SPLIT,:,:,:] #Reduced range to [0,1]
train_weights_w = weights[:DATA_SPLIT,:,:,:] #Reduced range to [0,1]
valid_weights_w = images[DATA_SPLIT:,:,:,:]
# train_weights_w = np.reshape(train_weights_w, (256,256,1,-1))

# U-Net
Based on this paper: [U-Net: Convolutional Networks for Biomedical
Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf).

In [ ]:
model = Unet_Vanilla()
model.compile()

In [ ]:
model.train([train_X], [train_Y], batch_size = 32, epochs = 100, validation_data=(valid_X, valid_Y))

# U-Net w/ Pretrained Encoder

In [ ]:
model = Unet_VGG16()
model.compile()

In [ ]:
model.train([train_X], [train_Y], batch_size = 32, epochs = 100, validation_data=(valid_X, valid_Y))

In [ ]:
model.unfreeze_encoder(optimizer=optimizers.Adam(0.0003))
model.train([train_X], [train_Y], batch_size = 32, epochs = 100, validation_data=(valid_X, valid_Y))

# Weighted Unet

In [ ]:
model = Unet_VGG16_Weighted()
model.compile()

In [ ]:
model.train(x=[train_X,train_weights_w], y=[train_Y], batch_size = 32, epochs = 100)

In [ ]:
model.unfreeze_encoder(optimizer=optimizers.Adam(0.0003))
model.train(x=[train_X,train_weights_w], y=[train_Y], batch_size = 32, epochs = 100)

# Make Predictions with Trained Model
In order to measure the accuracy of the model, I used a Intersection over Union (IOU) metric which utilizes the area of interection between the ground truth and the predicted mask, and compares it to the total area covered by the union of the two.

![iou](resources/iou_image.png)

In [ ]:
labels = pickle.load(open('../processed_data/train_Y_baseline.p', 'rb'))
labels = np.expand_dims(labels, axis=-1)
valid_Y = labels[DATA_SPLIT:,:,:,:] #Reduced range to [0,1]

In [ ]:
  # Predict Validation Data
pred = model2.predict([valid_X, np.zeros(shape=(len(valid_X),256,256,1))])
print(np.mean(pred))

In [ ]:
  # Predict Validation Data
pred = model2.predict(valid_X)
print(np.mean(pred))

In [ ]:
np.min(pred)

In [ ]:
sample = 1

fig, ax = plt.subplots(nrows=2,ncols=2)

pred_rounded = np.where(pred[sample,:,:,0] > 0.99, 1, 0)
ax[0,0].imshow(valid_X[sample,:,:,:])
ax[0,1].imshow(valid_Y[sample,:,:,0])
ax[1,0].imshow(pred[sample,:,:,0])
ax[1,1].imshow(pred_rounded)
np.amax(pred[sample,:,:,0])
# pred[sample,:,:,0]


In [ ]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = label(y_true_in == 1)
    y_pred = label(y_pred_in > 0.999)
    
    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = len(y_true_in)
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
#     return np.array(np.mean(metric), dtype=np.float32)
    return metric

In [ ]:
valid_Y[0,:,:,0]

In [ ]:
pred.shape

In [ ]:
iou_metric(valid_Y[:,:,:,0], pred[:,:,:,0])

In [ ]:
import cv2

In [ ]:
kernel = np.ones((10,10),np.uint8)
pred = pred.reshape(-1,256,256)
for i in range(len(pred)):
    pred[i] = cv2.morphologyEx(pred[i], cv2.MORPH_CLOSE, kernel)
pred = pred.reshape(-1,256,256,1)

In [ ]:
plt.imshow(pred[4][:,:,0])

## Separating Clusters
The output of the Fully Convolutional Network, while outputting a relatively accurate mask, fails to separate clusters of nuclei into individual instances. One option to solve this is to utilize traditional computer vision methods. One such method is Watershed which essentially shrinks images around points which are guaranteed to be the foreground, or part of the nuclei. An example of this is shown below:

![watershed](resources/watershed.png)

In [ ]:
def apply_watershed(pred):
    watershed_pred = []
    for i in range(len(pred)):
        image = pred[i,:,:,0] > 0.999
        
        
        distance = ndi.distance_transform_edt(image)
        local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((2, 2)),
                            labels=image)
        markers = ndi.label(local_maxi)[0]
        labels = watershed(-distance, markers, mask=image)
       
        watershed_pred.append(labels)
        
    watershed_pred = np.stack(watershed_pred, axis=0)
    return watershed_pred

watershed_pred = apply_watershed(pred)

In [ ]:
iou_metric(valid_Y[:,:,:,0], watershed_pred)

In [ ]:
print(np.amax(np.where(watershed_pred[0] >= 1, 1 ,0)))

In [ ]:
plt.imshow(valid_Y[2,:,:,0])

In [ ]:
plt.imshow(watershed_pred[2])

In [ ]:
plt.imshow(watershed_pred[0] - np.where(pred[0,:,:,0] > 0.55, 1,0))

In [ ]:
plt.imshow(valid_Y[1,:,:,0])
plt.figure()
plt.imshow(np.where(pred[1,:,:,0] > 0.55, 1,0))
plt.figure()
plt.imshow(watershed_pred[1])

## Methods for Saving/Loading Trained Model

### Save Model

In [ ]:
#SAVE MODEL

model2.save('../unet_gray.h5')

### Load Model

In [ ]:

from keras.models import load_model
# model = load_model('unet_baseline.h5', custom_objects={'weighted_binary_crossentropy_loss': weighted_binary_crossentropy_loss, 'mean_iou': mean_iou})
model2 = load_model('../unet.h5', custom_objects={'weighted_binary_crossentropy_loss': weighted_binary_crossentropy_loss, 'mean_iou': mean_iou})



## Use Test Data
Using the final pipeline, I predicted the masks for the test class, resized them to the original dimensions, and format the results into a submission file.

In [ ]:
test_images = pickle.load(open('../processed_data/test_X.p', 'rb'))
##test_images2 = pickle.load(open('test_X_baseline2.p', 'rb'))

##test_images = np.concatenate([test_images1, test_images2], axis=0) 

# test_sizes = pickle.load(open('../test_sizes.p', 'rb'))
# test_ids = pickle.load(open('../test_img_names.p', 'rb'))

In [ ]:
model = model2

In [ ]:
test_images = test_images - IMAGE_MEAN
pred = model.predict(test_images)
# watershed_pred = apply_watershed(pred)

In [ ]:

plt.imshow(test_images[sample,:,:,0])
plt.figure()
plt.imshow(pred[sample,:,:,0])
mask2 = np.where(pred[sample,:,:,0] > 0.999, 1, 0)
plt.figure()
plt.imshow(mask2)

In [ ]:
preds = []
for i in range(len(test_sizes)):
    preds.append(imresize(watershed_pred[i],test_sizes[i][:2]))

In [ ]:
plt.imshow(preds[0][:,:])

In [ ]:
import pandas as pd

In [ ]:

import re


def regex(txt):

  re1 = '(?:[a-z0-9][a-z]*[0-9]+[a-z0-9]*)/'

  rg = re.compile(re1)
  m = rg.search(txt)
  if m:
      alphanum1=m.group(0)
  return alphanum1[0:-1]


In [ ]:
sample = 2
shape = test_sizes[sample]
shape = (shape[0],shape[1])
mask = imresize(pred[sample,:,:,0], shape) / 255
mask2 = label(np.where(mask > 0.9, 1, 0),connectivity=2)



fig, ax = plt.subplots(nrows=2,ncols=3)

ax[0,0].imshow(test_images[sample])
ax[0,1].imshow(mask)
ax[0,2].imshow(mask2)

print(test_images[sample].shape)
print(mask.shape)

In [ ]:
#Separate Masks
from scipy import ndimage
labels, nlabels = ndimage.label(watershed[0])

label_arrays = []
for label_num in range(1, nlabels+1):
    label_mask = np.where(labels == label_num, 1, 0)
    label_arrays.append(label_mask)

print('There are {} separate components / objects detected.'.format(nlabels))

In [ ]:
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cut_off = 0.5):
    lab_img = label(x>cut_off)
    if lab_img.max()<1:
        lab_img[0,0] = 1 # ensure at least one prediction per image
    for i in range(1, lab_img.max()+1):
        yield rle_encoding(lab_img==i)

In [ ]:
new_test_ids = []
rles = []
count = 0
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds[n]))
    
    rles.extend(rle)
    new_test_ids.extend([id_[:-4] for i in range(len(rle))])

In [ ]:

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub[['ImageId', 'EncodedPixels']].to_csv('baseline_submission.csv', index=False)

In [ ]:
sample = pd.read_csv("stage2_sample_submission_final.csv")

In [ ]:
solution = sorted(zip(new_test_ids, rles))
new_test_ids = []
rles = []
for x,y in solution:
    new_test_ids.append(x)

    rles.append(y)

In [ ]:
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('baseline_submission.csv', index=False)

In [ ]:
len(sub.ImageId.unique())